# h2o benchmarks - query 7

In [6]:
import coiled
import dask
import dask.dataframe as dd

In [7]:
cluster = coiled.Cluster(name="powers-crt-004", software="crt-004", n_workers=5)

Output()

In [8]:
client = dask.distributed.Client(cluster)

## 5 GB dataset

In [20]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/h2o-benchmark/N_1e8_K_1e2_parquet",
    storage_options={"anon": True, "use_ssl": True},
    columns=["id3", "v1", "v2"],
)

In [10]:
%%time
ddf.groupby("id3", dropna=False, observed=True).agg({"v1": "max", "v2": "min"}).assign(
    range_v1_v2=lambda x: x["v1"] - x["v2"]
)[["range_v1_v2"]].compute()

CPU times: user 887 ms, sys: 172 ms, total: 1.06 s
Wall time: 1min 11s


,range_v1_v2
id3,
id0000608844,4
id0000466449,4
id0000573987,4
id0000776204,4
id0000608718,4
...,...
id0000821599,4
id0000937302,4
id0000248458,4


In [11]:
len(ddf)

100000000

In [21]:
ddf.npartitions

100

In [22]:
%%time
ddf.groupby("id3", dropna=False, observed=True).agg({"v1": "max", "v2": "min"}, split_out=2).assign(
    range_v1_v2=lambda x: x["v1"] - x["v2"]
)[["range_v1_v2"]].compute()

CPU times: user 4.98 s, sys: 655 ms, total: 5.64 s
Wall time: 9min 47s


,range_v1_v2
id3,
id0000573987,4
id0000776204,4
id0000545596,4
id0000339296,4
id0000226214,4
...,...
id0000710890,4
id0000549495,4
id0000937302,4


In [23]:
%%time
ddf.groupby("id3", dropna=False, observed=True).agg({"v1": "max", "v2": "min"}, split_out=20).assign(
    range_v1_v2=lambda x: x["v1"] - x["v2"]
)[["range_v1_v2"]].compute()

KeyboardInterrupt: 

## 50 GB dataset

In [15]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/h2o-benchmark/N_1e9_K_1e2_parquet",
    storage_options={"anon": True, "use_ssl": True},
    columns=["id3", "v1", "v2"],
)

In [16]:
%%time
ddf.groupby("id3", dropna=False, observed=True).agg({"v1": "max", "v2": "min"}).assign(
    range_v1_v2=lambda x: x["v1"] - x["v2"]
)[["range_v1_v2"]].compute()

KilledWorker: ("('aggregate-combine-9f834b1fe612b47d1cf674ecae566ac2', 1, 1)", <WorkerState 'tls://10.4.1.61:45373', name: powers-crt-004-worker-97c632c32a, status: closed, memory: 0, processing: 372>)

In [17]:
len(ddf)

1000000000

In [18]:
ddf.npartitions

2000

In [19]:
%%time
ddf.groupby("id3", dropna=False, observed=True).agg({"v1": "max", "v2": "min"}, split_out=2).assign(
    range_v1_v2=lambda x: x["v1"] - x["v2"]
)[["range_v1_v2"]].compute()

KeyboardInterrupt: 